<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/sd_diffusers_colab_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #Install dependencies
!git clone https://github.com/oneir0mancer/stable-diffusion-diffusers-colab-ui.git StableDiffusionUi

!pip install -r StableDiffusionUi/requirements.txt
!wget https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml

!mkdir -p outputs/{txt2img,img2img}
!git clone https://huggingface.co/embed/negative /content/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/Lora/positive


# Creating model pipeline


<details>
<summary><b>Description</b><br>TLDR: use dropdown or just paste stuff</summary>

---

You can just choose a model from a dropdown of popular models, which will also give you a link to a model page on [huggingface](https://huggingface.co) and any trigger words you need to use in the prompt.

Alternatively, you can paste a **model_id**, **url**, or **local path** for any model from huggingface:

* **model_id** looks like this: `gsdf/Counterfeit-V2.5`.
<br>You can use it for any *proper* diffusers model in huggingface, i.e. ones that have a lot of folders in their **Files**, and a file called **model_index.json**
<br> `🔲 A1111 format` toggle should be **<font color= 'red'>off</font>**.

* **url** from HuggingFace may look like this:
```
https://huggingface.co/WarriorMama777/OrangeMixs/blob/main/Models/AbyssOrangeMix/AbyssOrangeMix.safetensors
```
Use it when the model is in Automatic1111 format (as in, just a single file).
<br> `✅ A1111 format` toggle should be **<font color= 'green'>on</font>**.

* **url** from CivitAI should look like this:
```
https://civitai.com/api/download/models/XXXXXX
```
(the link you get from "Download" button)
<br> `✅ A1111 format` toggle should be **<font color= 'green'>on</font>**.

* **local path** is just a path to a model folder (if diffusers format) or model file (if A1111 format).
<br>Set toggle accordingly.

</details>

<details>
<summary>Where to find urls</summary>

A good place to find model urls is  [camenduru colabs repo](https://https://github.com/camenduru/stable-diffusion-webui-colab), just look for a line starting with `!aria2c ...`

</details>

In [ ]:
#@title Render model choice UI

from StableDiffusionUi.ColabUI.ColabWrapper import ColabWrapper

colab = ColabWrapper("outputs/txt2img")
colab.render_model_index("/content/StableDiffusionUi/model_index.json")

In [ ]:
#@title Load chosen SD model

#TODO we have to use this until LPW properly supports clip skip
from StableDiffusionUi.lpw_stable_diffusion import StableDiffusionLongPromptWeightingPipeline
colab.load_model(StableDiffusionLongPromptWeightingPipeline, custom_pipeline=None)

In [ ]:
#@title (*Optional*) Run this instead to load SDXL model
#@markdown You probably won't be able to load model in A1111 format because there is not enought RAM for conversion
from diffusers import StableDiffusionXLPipeline

colab.load_model(StableDiffusionXLPipeline, custom_pipeline="lpw_stable_diffusion_xl")

<details>
<summary>Prompting detail</summary>

I use [this](https://github.com/huggingface/diffusers/tree/main/examples/community#long-prompt-weighting-stable-diffusion) custom pipeline which doesn't have a token length limit and allows to use weights in prompt.

</details>

# Generating images

In [ ]:
#@title Render UI
#@markdown ---
from  StableDiffusionUi.ColabUI.DiffusionPipelineUI import DiffusionPipelineUI

ui = DiffusionPipelineUI()
colab.render_generation_ui(ui)

In [ ]:
#@title <font color='red'>Run</font> this cell to generate images
colab.generate()

#Utils

In [ ]:
#@title Img2img refiner UI
# We have to load custom pipeline ourselves because you can't pass it when reusing pipe components
from diffusers.utils import get_class_from_dynamic_module
Interface = get_class_from_dynamic_module(colab.custom_pipeline, module_file=f"{colab.custom_pipeline}.py")
colab.render_refiner_ui(Interface)

In [ ]:
#@title <font color='red'>Run</font> img2img
colab.refiner_generate(output_dir="outputs/img2img")

#Conversion

## Converting models from ckpt
You shouldn't really need this, because diffusers now support loading from A1111 format.

But if you want, you can download a model and convert it to diffusers format.

NOTE: A good place to find model urls is  [camenduru colabs repo](https://https://github.com/camenduru/stable-diffusion-webui-colab), just look for a line starting with `!aria2c ...`


In [ ]:
#@markdown ##Download ckpt file
#@markdown Download ckpt/safetensors file from huggingface url.

#@markdown ---
import os

url = "https://huggingface.co/mekabu/MagicalMix_v2/resolve/main/MagicalMix_v2.safetensors" #@param {type:"string"}
ckpt_dump_folder = "/content/models_sd/"    #@param {type:"string"}
model_name = url.split('/')[-1]

os.system(f"mkdir -p {ckpt_dump_folder}")
bashCommand = f"wget {url} -P {ckpt_dump_folder} --content-disposition"
os.system(bashCommand)

In [ ]:
#@markdown ##(Optional) Or just upload file
from google.colab import files
f = files.upload()

ckpt_dump_folder = "/content/"
for key in f.keys():
    model_name = key
    break

In [ ]:
#@markdown ##Run conversion script
#@markdown Paste a path to where you want this script to cache a model into `dump_path`.

#@markdown Keep `override_path` empty unless you uploaded your file to some custom directory.

import os
import torch
from diffusers.pipelines.stable_diffusion.convert_from_ckpt import download_from_original_stable_diffusion_ckpt

from_safetensors = True #@param {type:"boolean"}
override_path = "" #@param {type:"string"}
if override_path != "":
    checkpoint_path = override_path
else:
    checkpoint_path = os.path.join(ckpt_dump_folder, model_name)

pipe = download_from_original_stable_diffusion_ckpt(
        checkpoint_path=checkpoint_path,
        original_config_file = "/content/v1-inference.yaml",
        from_safetensors=from_safetensors,
    )

dump_path="models/ModelName/" #@param {type:"string"}
pipe.save_pretrained(dump_path, safe_serialization=from_safetensors)

pipe = pipe.to("cuda")
pipe.safety_checker = None
pipe.to(torch_dtype=torch.float16)

## Converting VAE

In [ ]:
#@markdown ##Download vae file
#@markdown Basically, the same thing as model files

#@markdown ---
import os

url = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt" #@param {type:"string"}
ckpt_dump_folder = "/content/models_sd/"    #@param {type:"string"}
model_name = url.split('/')[-1]

os.system(f"mkdir -p {ckpt_dump_folder}")
bashCommand = f"wget {url} -P {ckpt_dump_folder} --content-disposition"
os.system(bashCommand)

In [ ]:
#@markdown Paste a path to where you want this script to cache a vae into `dump_path`.
from_safetensors = False #@param {type:"boolean"}
dump_path="vae/VaeName/" #@param {type:"string"}

checkpoint_path = os.path.join(ckpt_dump_folder, model_name)
bashCommand = f"python /content/StableDiffusionUi/scripts/convert_vae_pt_to_diffusers.py --vae_pt_path {checkpoint_path} --dump_path {dump_path}"
if from_safetensors:
    bashCommand += " --from_safetensors"
os.system(bashCommand)

#<font color='red'>Flush</font> model


In [ ]:
import gc
import torch

del colab.pipe
torch.cuda.empty_cache()
gc.collect()